# Example: Loading a Trained NatPN Model

In this example, we want to use one of the pretrained models that are available on our [Weights & Biases project](https://wandb.ai/borchero/natural-posterior-network).

Although the following is not required, we first want to hide some useless output generated by PyTorch Lightning.

In [1]:
from natpn import suppress_pytorch_lightning_logs
suppress_pytorch_lightning_logs()

/Users/borchero/github/natural-posterior-network/.venv/lib/python3.9/site-packages/torchvision/io/image.py:9: UserWarning: Failed to load image Python extension: 
  warn(f"Failed to load image Python extension: {e}")


## Load Model

Then, we can load the model via the Weights & Biases API. For this example, we choose a training run  on the Sensorless Drive dataset, see [here](https://wandb.ai/borchero/natural-posterior-network/runs/4h6exog8).

In [2]:
import wandb

run = wandb.init()
artifact = run.use_artifact("borchero/natural-posterior-network/sensorless-drive:v0", type="model")
artifact_dir = artifact.download()

wandb: Currently logged in as: borchero (use `wandb login --relogin` to force relogin)


In [3]:
from natpn import NaturalPosteriorNetwork

estimator = NaturalPosteriorNetwork.load(artifact_dir)

After the estimator has been loaded, we can access its fitted attributes: for example, the trained model.

In [4]:
estimator.model_

NaturalPosteriorNetworkModel(
  (encoder): TabularEncoder(
    (0): Linear(in_features=48, out_features=64, bias=True)
    (1): LeakyReLU(negative_slope=0.01)
    (2): Dropout(p=0.0, inplace=False)
    (3): Linear(in_features=64, out_features=64, bias=True)
    (4): LeakyReLU(negative_slope=0.01)
    (5): Dropout(p=0.0, inplace=False)
    (6): Linear(in_features=64, out_features=64, bias=True)
    (7): LeakyReLU(negative_slope=0.01)
    (8): Dropout(p=0.0, inplace=False)
    (9): Linear(in_features=64, out_features=4, bias=True)
  )
  (flow): RadialFlow(
    (transforms): ModuleList(
      (0): RadialTransform()
      (1): RadialTransform()
      (2): RadialTransform()
      (3): RadialTransform()
      (4): RadialTransform()
      (5): RadialTransform()
      (6): RadialTransform()
      (7): RadialTransform()
    )
  )
  (output): CategoricalOutput(
    (linear): Linear(in_features=4, out_features=9, bias=True)
    (prior): DirichletPrior()
  )
  (scaler): EvidenceScaler()
)

## Evaluate Estimator

Using the fitted estimator, we can, of course, also evaluate its performance. For obtaining the same scores that are stored on Weights & Biases, we need to take care to pass the training run's seed to the initializer of the data module. This way, we ensure that train/test splits stay consistent.

In [5]:
from natpn.datasets import SensorlessDriveDataModule

In [6]:
dm = SensorlessDriveDataModule(seed=42)

In [7]:
estimator.score(dm)

Evaluating on test set...
Preparing 'Sensorless Drive'...


Testing: 0it [00:00, ?it/s]

{'accuracy': 0.9940463900566101, 'brier_score': 0.04772574454545975}

In [8]:
estimator.score_ood_detection(dm)

Evaluating in-distribution vs. sensorless_drive_left_out...


Testing: 0it [00:00, ?it/s]

Evaluating in-distribution vs. sensorless_drive_oodom...


Testing: 0it [00:00, ?it/s]

{'sensorless_drive_left_out': {'aleatoric_confidence_auc_pr': 0.6704671382904053,
  'aleatoric_confidence_auc_roc': 0.6916689276695251,
  'epistemic_confidence_auc_pr': 0.6039667129516602,
  'epistemic_confidence_auc_roc': 0.6485900282859802},
 'sensorless_drive_oodom': {'aleatoric_confidence_auc_pr': 0.9998626112937927,
  'aleatoric_confidence_auc_roc': 0.999738872051239,
  'epistemic_confidence_auc_pr': 1.0,
  'epistemic_confidence_auc_roc': 1.0000001192092896}}